# Initial Test

* Aluno: Artur Chiaperini Grover
* Exchange: CEX ([CEX.IO](https://cex.io))

In [1]:
import pickle
import pandas as pd
import numpy as np
import ccxt
import datetime
import requests
import json
import time
import sqlite3
import sqlalchemy
from IPython.display import clear_output

----
## Function definition

In [2]:
def create_date_list(start, end):
    '''
    create_date_list(start, end):
    returns a list of dates from start to end with format 'YYYYMMDD' and type string.
    
    parameters:
    start -> Starting date with format 'YYYY-MM-DD' (string).
    end   -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    step = datetime.timedelta(days = 1)
    date_list = list()
    while start <= end:
        date_list.append(str(start.date()).replace('-', ''))
        start += step
        
    return date_list

In [3]:
def create_url_list(symbol1, symbol2, start, end):
    '''
    create_url_list(symbol1, symbol2, start, end):
    returns a list of urls from start to end date of the symbol1/symbol2 transaction.
    
    parameters:
    symbol1 -> Cryptocurrency acronym (string).
    symbol2 -> Currency acronym (string).
    start   -> Starting date with format 'YYYY-MM-DD' (string).
    end     -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    
    date_list = create_date_list(start, end)
    url_list = list()
    for date in date_list:
        url = 'http://cex.io/api/ohlcv/hd/{}/{}/{}'.format(date, symbol1, symbol2)
        url_list.append(url)
        
    return url_list

In [31]:
def get_ohlcv(symbol1, symbol2, start, end, data_rate):
    '''
    get_ohlcv(symbol1, symbol2, start, end, data_rate):
    returns a string with all read fetched data from cex.io for the given pair symbol1/symbol2 between
    the starting and ending dates and with data rate (1m, 1h or 1d).
    
    Function has a one second sleep to avoid exploding rate limit of exchange (cex.io).
    Cex rate limit is 600 requests per 10 minutes (1 request per second).
    
    parameters:
    symbol1   -> Cryptocurrency acronym (string).
    symbol2   -> Currency acronym (string).
    start     -> Starting date with format 'YYYY-MM-DD' (string).
    end       -> Ending date with format 'YYYY-MM-DD' (string).
    data_rate -> Data rate, cex.io gives the following options: 'data1m', 'data1h' or 'data1d'.
    '''
    
    url_list = create_url_list(symbol1, symbol2, start, end)
    # time_sleep: guarantees that no more than 1 request are done per second.
    time_sleep = 1.12
    print('Number of urls = {}'.format(len(url_list)))
    print('Time sleep = {}'.format(time_sleep))
    count_input = 0
    data_ohlcv_list = []
    for url in url_list:
        print('Reading {} -----'.format(url))
        request_url = requests.get(url).json()
        if request_url == None:
            print('No data to fetch... -----')
            pass
        else:
            ohlcv = eval(requests.get(url).json()[data_rate])
            data_ohlcv_list.append(ohlcv)
            print('Fetch input data size {}'.format(len(ohlcv)))
            count_input += len(ohlcv)
            print('Finished -----')
        
        time.sleep(time_sleep)
        
    print('Total number of inputs {}'.format(count_input))
    return str([item for sublist in data_ohlcv_list for item in sublist])

----
## Fetching data from Exchange and Saving Pickle File

### Etherium

In [5]:
# %%time
# cex_eth_usd = get_ohlcv('ETH', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [6]:
# file_ohlcv_eth_usd = 'ohlcv_eth_usd.pkl'
# pickle.dump(cex_eth_usd, open(file_ohlcv_eth_usd, 'wb'))

In [7]:
# %%time
# cex_eth_eur = get_ohlcv('ETH', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [8]:
# file_ohlcv_eth_eur = 'ohlcv_eth_eur.pkl'
# pickle.dump(cex_eth_eur, open(file_ohlcv_eth_eur, 'wb'))

### Bitcoin

In [9]:
# %%time
# cex_btc_usd = get_ohlcv('BTC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [10]:
# file_ohlcv_btc_usd = 'ohlcv_btc_usd.pkl'
# pickle.dump(cex_btc_usd, open(file_ohlcv_btc_usd, 'wb'))

In [9]:
# %%time
# cex_btc_eur = get_ohlcv('BTC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [10]:
# file_ohlcv_btc_eur = 'ohlcv_btc_eur.pkl'
# pickle.dump(cex_btc_eur, open(file_ohlcv_btc_eur, 'wb'))

### ZEC - Zcash

In [33]:
%%time
cex_zec_usd = get_ohlcv('ZEC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

Number of urls = 366
Time sleep = 1.12
Reading http://cex.io/api/ohlcv/hd/20170122/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170123/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170124/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170125/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170126/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170127/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170128/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170129/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170130/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170131/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170201/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20

No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170430/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170501/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170502/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170503/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170504/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170505/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170506/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170507/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170508/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170509/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170510/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170511/ZEC/US

No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170806/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170807/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170808/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170809/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170810/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170811/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170812/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170813/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170814/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170815/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170816/ZEC/USD -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170817/ZEC/US

Fetch input data size 85
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171101/ZEC/USD -----
Fetch input data size 142
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171102/ZEC/USD -----
Fetch input data size 153
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171103/ZEC/USD -----
Fetch input data size 207
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171104/ZEC/USD -----
Fetch input data size 90
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171105/ZEC/USD -----
Fetch input data size 88
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171106/ZEC/USD -----
Fetch input data size 152
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171107/ZEC/USD -----
Fetch input data size 164
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171108/ZEC/USD -----
Fetch input data size 231
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171109/ZEC/USD -----
Fetch input data size 291
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171110/ZEC/USD -----
Fetch input d

Fetch input data size 436
Finished -----
Total number of inputs 49601
CPU times: user 14 s, sys: 1.21 s, total: 15.2 s
Wall time: 14min 53s


In [34]:
file_ohlcv_zec_usd = 'ohlcv_zec_usd.pkl'
pickle.dump(cex_zec_usd, open(file_ohlcv_zec_usd, 'wb'))

In [35]:
%%time
cex_zec_eur = get_ohlcv('ZEC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

Number of urls = 366
Time sleep = 1.12
Reading http://cex.io/api/ohlcv/hd/20170122/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170123/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170124/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170125/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170126/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170127/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170128/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170129/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170130/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170131/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170201/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20

No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170430/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170501/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170502/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170503/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170504/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170505/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170506/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170507/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170508/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170509/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170510/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170511/ZEC/EU

No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170806/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170807/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170808/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170809/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170810/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170811/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170812/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170813/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170814/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170815/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170816/ZEC/EUR -----
No data to fetch... -----
Reading http://cex.io/api/ohlcv/hd/20170817/ZEC/EU

Fetch input data size 31
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171101/ZEC/EUR -----
Fetch input data size 40
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171102/ZEC/EUR -----
Fetch input data size 28
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171103/ZEC/EUR -----
Fetch input data size 63
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171104/ZEC/EUR -----
Fetch input data size 38
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171105/ZEC/EUR -----
Fetch input data size 28
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171106/ZEC/EUR -----
Fetch input data size 32
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171107/ZEC/EUR -----
Fetch input data size 35
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171108/ZEC/EUR -----
Fetch input data size 29
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171109/ZEC/EUR -----
Fetch input data size 40
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171110/ZEC/EUR -----
Fetch input data siz

Total number of inputs 11720
CPU times: user 13.2 s, sys: 1.29 s, total: 14.5 s
Wall time: 14min 34s


In [36]:
file_ohlcv_zec_eur = 'ohlcv_zec_eur.pkl'
pickle.dump(cex_zec_eur, open(file_ohlcv_zec_eur, 'wb'))

----
## Renaming and Correcting Datetime

In [11]:
loaded_ohlcv_eth_usd = pickle.load(open('ohlcv_eth_usd.pkl', "rb"))

In [12]:
df = pd.read_json(loaded_ohlcv_eth_usd)
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [13]:
format_date = lambda x: datetime.datetime.fromtimestamp(x) 
df['date'] = pd.to_datetime(df['date'].apply(format_date))

In [14]:
df.set_index('date', inplace = True)

In [15]:
df

,open,high,low,close,volume
date,,,,,
2017-01-21 23:14:00,10.800000,10.800000,10.800000,10.800000,21.317518
2017-01-21 23:55:00,10.960000,10.960000,10.960000,10.960000,2.134124
2017-01-22 00:02:00,10.850000,10.850000,10.850000,10.850000,9.971791
2017-01-22 00:07:00,10.850000,10.850000,10.850000,10.850000,6.622109
2017-01-22 00:08:00,10.850000,10.850000,10.850000,10.850000,9.882933
2017-01-22 00:11:00,10.850000,10.850000,10.850000,10.850000,100.000000
2017-01-22 00:13:00,10.850000,10.850000,10.850000,10.850000,3.603681
2017-01-22 00:21:00,10.850000,10.850000,10.850000,10.850000,3.808289
2017-01-22 00:29:00,10.850000,10.850000,10.850000,10.850000,46.000000
